In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from scipy.stats.stats import pearsonr
# import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.spatial.distance import pdist, squareform
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections


In [7]:
songs = pd.read_csv("data/small_dbsongs_all_b.csv", index_col=0)
songs = songs.drop_duplicates()
songs = songs.reset_index()

In [8]:
songs.head()

,index,song_id,audio_md5,bars_confidence,beats_start,danceability,duration,energy,key,loudness,...,segments_pitches,segments_start,segments_timbre,song_hotttnesss,tempo,artist_name,artist_playmeid,artist_terms,artist_terms_freq,artist_terms_weight
0,0,SOLDPNN12A6D4F7B46,f890315e2534fa092566cc8cdeb7ef7b,"[0.752, 0.542, 0.365, 0.301, 0.513, 0.781, 0.7...","[0.23787, 0.59778, 0.9488, 1.30427, 1.65795, 2...",0.0,308.45342,0.0,2,-11.082,...,"[[0.344, 0.576, 0.664, 0.98, 0.846, 1.0, 0.898...","[0.0, 0.40336, 0.75896, 1.10544, 1.47156, 1.79...","[[28.948, -81.153, 33.662, -1.596, -22.445, 45...",NaN,168.932,Chris Rea,-1,"[""soft rock"", ""blues-rock"", ""pop rock"", ""count...","[0.9613752143518429, 0.9091657447206882, 0.977...","[1.0, 0.9846244162566241, 0.9734501260825714, ..."
1,1,SOVSNJX12AB0189848,e6e3aa56ffec8b14e3cbc0329048820f,"[0.009, 0.076, 0.012, 0.082, 0.007, 0.044, 0.0...","[0.22693, 0.6456, 1.05559, 1.46408, 1.87154, 2...",0.0,216.08444,0.0,9,-5.892,...,"[[0.269, 0.925, 1.0, 0.426, 0.115, 0.095, 0.06...","[0.0, 0.60676, 1.02372, 1.23878, 1.4395, 1.632...","[[9.183, 122.636, 44.109, -223.254, 20.523, 6....",0.355286,147.944,BoDeans,20696,"[""roots rock"", ""heartland rock"", ""pop rock"", ""...","[0.9770567375665963, 0.9618410956772863, 0.969...","[1.0, 0.9882147281342009, 0.9134140063765218, ..."
2,2,SOAIEFA12AC9097225,d88f29ef5bdd39c0bea260f2ad8ac511,"[0.25, 0.26, 0.037, 0.062, 0.069, 0.122, 0.118...","[0.36869, 0.86525, 1.35457, 1.83305, 2.32011, ...",0.0,265.71710,0.0,5,-6.409,...,"[[0.152, 0.39, 0.495, 0.345, 0.75, 1.0, 0.492,...","[0.0, 0.279, 0.83991, 1.30844, 2.04132, 2.3074...","[[5.284, 95.237, 46.058, -152.53, 44.608, -68....",NaN,124.448,Black Debbath,-1,"[""blues"", ""metal"", ""comedy"", ""classic rock"", ""...","[1.0, 0.8491785173847317, 0.6595623713319314, ...","[1.0, 0.8057140351684267, 0.8056652592724313, ..."
3,3,SOMAGRE12A6D4F8C60,d21275c973cfc35ba12c7a6f8887ff3f,"[0.164, 0.175, 0.393, 0.19, 0.208, 0.847, 0.05...","[0.24211, 1.20848, 2.17484, 3.13875, 4.10758, ...",0.0,205.92281,0.0,2,-13.522,...,"[[1.0, 0.883, 0.622, 0.369, 0.229, 0.13, 0.256...","[0.0, 0.26236, 0.35633, 0.88825, 0.9615, 1.088...","[[0.471, 161.108, -5.1, -41.252, 46.182, -51.1...",0.623480,61.587,John Ottman,-1,"[""acid jazz"", ""jazz"", ""gangster rap"", ""classic...","[0.9946621853882806, 0.9973867110313376, 0.793...","[1.0, 0.8953389273306556, 0.8885069479563394, ..."
4,4,SOTDYKK12AB0184B0B,8089e6b549da324b33b528c38f5276b0,"[0.035, 0.085, 0.307, 0.111, 0.17, 0.04, 0.018...","[0.30076, 0.64684, 0.99207, 1.33645, 1.68422, ...",0.0,263.52281,0.0,4,-12.732,...,"[[0.214, 0.176, 0.182, 0.542, 1.0, 0.301, 0.19...","[0.0, 0.26404, 0.60127, 1.52925, 1.63429, 1.98...","[[17.883, 25.454, 38.934, -267.713, 11.12, 50....",0.496175,178.299,The Black Crowes,6981,"[""blues-rock"", ""southern rock"", ""hard rock"", ""...","[0.9009343605424817, 0.9009343605424817, 0.901...","[1.0, 0.9910312430812491, 0.8634263471078816, ..."


In [9]:
import csv 
fpath = "data/small_dbsongs_all_b.csv"
with open(fpath, 'r') as infile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames
fieldnames

['',
 'song_id',
 'audio_md5',
 'bars_confidence',
 'beats_start',
 'danceability',
 'duration',
 'energy',
 'key',
 'loudness',
 'mode',
 'mode_confidence',
 'release',
 'sections_confidence',
 'sections_start',
 'segments_confidence',
 'segments_loudness_max_time',
 'segments_loudness_start',
 'segments_pitches',
 'segments_start',
 'segments_timbre',
 'song_hotttnesss',
 'tempo',
 'artist_name',
 'artist_playmeid',
 'artist_terms',
 'artist_terms_freq',
 'artist_terms_weight']

In [10]:
print(songs[:5]['song_hotttnesss'])

0         NaN
1    0.355286
2         NaN
3    0.623480
4    0.496175
Name: song_hotttnesss, dtype: float64


In [11]:
songs['mode'][0]

0

In [12]:
songs = songs.drop(['artist_playmeid'], axis=1)
songs = songs.drop(["danceability"],axis=1) # all 0s
songs = songs.drop(["mode_confidence"],axis=1)
songs = songs.drop(["energy"],axis=1) # all 0s
songs = songs.drop(["release"], axis=1)
#songs = songs.drop(["year"],axis=1)
songs.head()

,index,song_id,audio_md5,bars_confidence,beats_start,duration,key,loudness,mode,sections_confidence,...,segments_loudness_start,segments_pitches,segments_start,segments_timbre,song_hotttnesss,tempo,artist_name,artist_terms,artist_terms_freq,artist_terms_weight
0,0,SOLDPNN12A6D4F7B46,f890315e2534fa092566cc8cdeb7ef7b,"[0.752, 0.542, 0.365, 0.301, 0.513, 0.781, 0.7...","[0.23787, 0.59778, 0.9488, 1.30427, 1.65795, 2...",308.45342,2,-11.082,0,"[1.0, 0.259, 0.04, 0.021, 0.522, 0.144, 0.87, ...",...,"[-47.21, -33.18, -34.054, -31.296, -31.877, -3...","[[0.344, 0.576, 0.664, 0.98, 0.846, 1.0, 0.898...","[0.0, 0.40336, 0.75896, 1.10544, 1.47156, 1.79...","[[28.948, -81.153, 33.662, -1.596, -22.445, 45...",NaN,168.932,Chris Rea,"[""soft rock"", ""blues-rock"", ""pop rock"", ""count...","[0.9613752143518429, 0.9091657447206882, 0.977...","[1.0, 0.9846244162566241, 0.9734501260825714, ..."
1,1,SOVSNJX12AB0189848,e6e3aa56ffec8b14e3cbc0329048820f,"[0.009, 0.076, 0.012, 0.082, 0.007, 0.044, 0.0...","[0.22693, 0.6456, 1.05559, 1.46408, 1.87154, 2...",216.08444,9,-5.892,1,"[1.0, 0.67, 0.791, 0.782, 0.635, 1.0, 0.425]",...,"[-60.0, -57.516, -17.528, -20.491, -20.991, -2...","[[0.269, 0.925, 1.0, 0.426, 0.115, 0.095, 0.06...","[0.0, 0.60676, 1.02372, 1.23878, 1.4395, 1.632...","[[9.183, 122.636, 44.109, -223.254, 20.523, 6....",0.355286,147.944,BoDeans,"[""roots rock"", ""heartland rock"", ""pop rock"", ""...","[0.9770567375665963, 0.9618410956772863, 0.969...","[1.0, 0.9882147281342009, 0.9134140063765218, ..."
2,2,SOAIEFA12AC9097225,d88f29ef5bdd39c0bea260f2ad8ac511,"[0.25, 0.26, 0.037, 0.062, 0.069, 0.122, 0.118...","[0.36869, 0.86525, 1.35457, 1.83305, 2.32011, ...",265.71710,5,-6.409,1,"[1.0, 0.412, 0.239, 0.1, 0.162, 0.104, 1.0, 0....",...,"[-60.0, -33.221, -13.795, -15.749, -14.917, -1...","[[0.152, 0.39, 0.495, 0.345, 0.75, 1.0, 0.492,...","[0.0, 0.279, 0.83991, 1.30844, 2.04132, 2.3074...","[[5.284, 95.237, 46.058, -152.53, 44.608, -68....",NaN,124.448,Black Debbath,"[""blues"", ""metal"", ""comedy"", ""classic rock"", ""...","[1.0, 0.8491785173847317, 0.6595623713319314, ...","[1.0, 0.8057140351684267, 0.8056652592724313, ..."
3,3,SOMAGRE12A6D4F8C60,d21275c973cfc35ba12c7a6f8887ff3f,"[0.164, 0.175, 0.393, 0.19, 0.208, 0.847, 0.05...","[0.24211, 1.20848, 2.17484, 3.13875, 4.10758, ...",205.92281,2,-13.522,1,"[1.0, 0.019, 0.303, 0.364, 0.581, 0.639, 0.188...",...,"[-60.0, -56.528, -54.01, -49.253, -48.218, -46...","[[1.0, 0.883, 0.622, 0.369, 0.229, 0.13, 0.256...","[0.0, 0.26236, 0.35633, 0.88825, 0.9615, 1.088...","[[0.471, 161.108, -5.1, -41.252, 46.182, -51.1...",0.623480,61.587,John Ottman,"[""acid jazz"", ""jazz"", ""gangster rap"", ""classic...","[0.9946621853882806, 0.9973867110313376, 0.793...","[1.0, 0.8953389273306556, 0.8885069479563394, ..."
4,4,SOTDYKK12AB0184B0B,8089e6b549da324b33b528c38f5276b0,"[0.035, 0.085, 0.307, 0.111, 0.17, 0.04, 0.018...","[0.30076, 0.64684, 0.99207, 1.33645, 1.68422, ...",263.52281,4,-12.732,0,"[1.0, 0.406, 0.385, 0.458, 0.142, 0.028, 0.438...",...,"[-60.0, -26.561, -24.673, -26.894, -22.663, -2...","[[0.214, 0.176, 0.182, 0.542, 1.0, 0.301, 0.19...","[0.0, 0.26404, 0.60127, 1.52925, 1.63429, 1.98...","[[17.883, 25.454, 38.934, -267.713, 11.12, 50....",0.496175,178.299,The Black Crowes,"[""blues-rock"", ""southern rock"", ""hard rock"", ""...","[0.9009343605424817, 0.9009343605424817, 0.901...","[1.0, 0.9910312430812491, 0.8634263471078816, ..."


In [13]:
songs = songs.replace(to_replace=np.NaN,value=0.00)

In [14]:
songs.describe()

,index,duration,key,loudness,mode,song_hotttnesss,tempo
count,856.000000,856.000000,856.000000,856.000000,856.000000,856.000000,856.000000
mean,470.021028,234.108812,5.302570,-10.453928,0.690421,0.191587,123.271626
std,282.669434,115.588901,3.648381,5.233128,0.462591,0.257058,34.790725
min,0.000000,7.130980,0.000000,-38.148000,0.000000,0.000000,0.000000
25%,230.750000,173.341587,2.000000,-13.083750,0.000000,0.000000,98.019500
50%,458.500000,219.049345,5.000000,-9.440000,1.000000,0.000000,120.124500
75%,701.250000,267.650155,9.000000,-6.494250,1.000000,0.405116,143.860750
max,998.000000,1280.887710,11.000000,0.566000,1.000000,1.000000,258.677000


In [19]:
q = songs

q = q._get_numeric_data()

print q.shape
print np.random.randn(5)

(856, 7)
[ 0.74342583 -0.0666618  -0.54070671  1.66420954  0.02890733]


In [18]:

# Dimension of our vector space
dimension = 500

# Create a random binary hash with 10 bits
rbp = RandomBinaryProjections('rbp', 10)

# Create engine with pipeline configuration
engine = Engine(q.shape[0], lshashes=[rbp])

# Index 1000000 random vectors (set their data to a unique string)
for index in range(100000):
    # v = np.random.randn(dimension)
    v = q 
    engine.store_vector(v, 'data_%d' % index)

# Create random query vector
query = np.random.randn(q.shape[0])

# Get nearest neighbours
N = engine.neighbours(query)

print N

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:

from pyspark import SparkContext
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import SQLContext

import findspark
findspark.init()

sc = SparkContext(appName="Recommender")

In [17]:
sqlContext = SQLContext(sc)
spDF = sqlContext.createDataFrame(q)
spDF.show()


+-----+------------------+---+-------------------+----+-------------------+------------------+
|index|          duration|key|           loudness|mode|    song_hotttnesss|             tempo|
+-----+------------------+---+-------------------+----+-------------------+------------------+
|    0|         308.45342|  2|            -11.082|   0|                0.0|168.93200000000004|
|    1|         216.08444|  9|             -5.892|   1| 0.3552855364230894|           147.944|
|    2|          265.7171|  5|             -6.409|   1|                0.0|           124.448|
|    3|         205.92281|  2|            -13.522|   1| 0.6234799017923877|            61.587|
|    4|         263.52281|  4|            -12.732|   0| 0.4961753356732933|           178.299|
|    5|         188.65587| 11|             -8.593|   1| 0.4173138028026521|           157.111|
|    6|          151.7971|  0|            -21.403|   1|0.24906579485370345|160.97899999999996|
|    7|         228.80608|  5|            -10.688|

In [22]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

import findspark
findspark.init()

import pyspark

sc = pyspark.SparkContext(appName="recommender")

# Trains a k-means model.
kmeans = KMeans().setK(10).setSeed(1)
model = kmeans.fit(q)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Exception AttributeError: "'NoneType' object has no attribute '_detach'" in <bound method KMeans.__del__ of KMeans_464fb8b19b2f9e9eb9a4> ignored


AttributeError: 'DataFrame' object has no attribute '_jdf'

In [41]:
kmeans_model = KMeans(n_clusters = 15, random_state = 1)
y_pred=kmeans_model.fit_predict(q)
center= kmeans_model.cluster_centers_


In [42]:
kmeans_model

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=1, tol=0.0001, verbose=0)

In [45]:
q = np.array(q)
q

array([[0.00000000e+00, 4.01997543e-01, 0.00000000e+00, ...,
        9.21980000e+01, 1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 4.17499645e-01, 1.00000000e+00, ...,
        1.21274000e+02, 1.00000000e+00, 0.00000000e+00],
       [2.00000000e+00, 3.43428378e-01, 0.00000000e+00, ...,
        1.00070000e+02, 1.00000000e+00, 5.65000000e-01],
       ...,
       [9.99700000e+03, 2.89902932e-01, 0.00000000e+00, ...,
        1.18123000e+02, 9.60115004e-01, 5.65000000e-01],
       [9.99800000e+03, 2.16828886e-01, 0.00000000e+00, ...,
        1.37663000e+02, 1.00000000e+00, 5.03000000e-01],
       [9.99900000e+03, 5.09243099e-01, 0.00000000e+00, ...,
        1.50575000e+02, 9.26219624e-01, 0.00000000e+00]])

In [43]:
user = pd.read_csv("data/kaggle_visible_evaluation_triplets.txt",nrows = 100000)

In [47]:
tab = lambda x: pd.Series([i for i in x.split('\t')])
user = user['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d	SOBONKR12A58A7A7E0	1'].apply(tab)

In [48]:
user = user.rename(index=str, columns={0: "userid", 1: "songid", 2 : "count"})

In [49]:
user.head()

,userid,songid,count
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOXLOQG12AF72A2D55,1


In [50]:
# Divide dataset into training data and testing dataset
# Training dataset is 80% and testing dataset is 20%

trainingdata = user.ix[:80000,:]
testingdata = user.ix[80000:100000,:]

In [51]:
trainingdata.head()

,userid,songid,count
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOXLOQG12AF72A2D55,1


In [52]:
testingdata.head()

,userid,songid,count
80000,af4773579e255b23589590ecad0648e27adf5f7f,SOWCKVR12A8C142411,1
80001,af4773579e255b23589590ecad0648e27adf5f7f,SOVFBUL12A58A7B498,1
80002,af4773579e255b23589590ecad0648e27adf5f7f,SOQUJLT12A8C141F8A,1
80003,af4773579e255b23589590ecad0648e27adf5f7f,SONYKOW12AB01849C9,16
80004,af4773579e255b23589590ecad0648e27adf5f7f,SODWUBY12A6D4F8E8A,1


In [54]:
song = trainingdata.merge(songs, left_on='songid', right_on='song.id', how='inner')

In [57]:
test = song.groupby('userid').mean()
test

,index,artist.hotttnesss,artist_mbtags_count,bars_confidence,bars_start,beats_confidence,beats_start,duration,end_of_fade_in,familiarity,...,longitude,loudness,release.id,song.hotttnesss,start_of_fade_out,tatums_confidence,tatums_start,tempo,terms_freq,finalmode
userid,,,,,,,,,,,,,,,,,,,,,
0011d5f4fb02ff276763d385c3f2ded2b00ad94a,3886.0,0.922412,1.0,0.003000,0.397490,0.655000,0.397490,245.367710,2.101000,0.889749,...,-63.933358,-4.536000,382330.0,0.853829,234.185000,0.743000,0.397490,160.7860,0.970217,0.676000
00a80e20a8b269747ad5f511fb579ef9ec013941,560.0,0.780334,9.0,0.082000,0.805450,0.482000,0.444130,252.342400,0.000000,0.852136,...,-63.933358,-3.028000,311757.0,0.801014,244.941000,0.155000,0.444130,167.4860,0.973315,0.462000
0107197bd5e2521cfa1cfde3faf9a6b39e8ba9e3,1781.0,0.713670,3.0,0.061000,1.378140,0.000000,0.351910,524.877910,0.708000,0.844085,...,-63.933358,-7.219000,15638.0,0.701040,500.018000,0.508000,0.100660,118.1010,0.938552,0.405000
0118d4558afda9bdaa39ec3ea3dbe62f12604d4d,2687.0,0.316853,1.0,0.243000,1.016680,0.505000,0.246520,411.689340,2.397000,0.612245,...,-63.933358,-7.172000,85226.0,0.270776,407.696000,0.205000,0.055420,156.0660,1.000000,0.340000
0160c5e9f3c439d601d379ea0d284e3979b4fc19,8604.0,0.554633,1.0,0.372000,0.052160,1.000000,0.052160,216.606890,0.000000,0.902063,...,-87.632410,-5.721000,186934.0,0.648463,201.201000,0.759000,0.052160,200.1140,1.000000,0.475000
026db54b18e6e1c8273d82f2f6b1f1c81676d01b,833.0,0.639798,0.0,0.034000,0.889970,0.652000,0.223800,417.671380,0.000000,0.830593,...,-73.948880,-7.865000,304844.0,0.653307,417.671000,0.402000,0.052700,181.5490,0.999946,0.289000
02caaa0349f5d7aa54a6a262308a4ada43ad46d2,6878.0,0.507464,1.0,0.029000,0.423840,1.000000,0.423840,211.695870,0.139000,0.754918,...,-2.632650,-7.750000,786795.0,0.864249,198.536000,0.988000,0.159650,113.3590,1.000000,0.434000
02e4af8e545c96c98359bc875a2db8f1e28bc42b,9734.0,0.579989,1.0,0.008000,1.361370,0.561000,0.683500,228.335870,4.180000,0.744641,...,-63.933358,-5.571000,7133.0,0.708108,206.065000,0.968000,0.342380,88.4710,0.999932,0.169000
03ace7391031faa7d9b152f39fe78cb57e0f078c,4139.0,0.335369,0.0,0.286000,0.187160,0.000000,0.187160,203.232200,0.160000,0.782556,...,-83.222950,-4.703000,605380.0,0.000000,200.121000,1.000000,0.187160,104.7100,0.992642,0.480000


In [58]:
test = test.dropna()

In [61]:
allusers = test.index

In [62]:
reco = []
count = 0
for i in allusers:
    currentuser = []
    currentuser.append(i)
    thisuser = list(test.loc[i])
    neigh = NearestNeighbors(n_neighbors=10)
    neigh.fit(q)
    NearestNeighbors(algorithm='ball_tree', leaf_size=100,metric='euclidean')
    distance, indices = neigh.kneighbors([thisuser])
    indices=list(indices[0])
    for ind in indices:
        currentuser.append(songs.ix[ind]["song.id"])
    count += 1
    reco.append(currentuser)

In [63]:
recommendations = pd.DataFrame(data = reco, columns = ["user_id","song1","song2","song3","song4","song5","song6","song7","song8","song9","song10"])
recommendations

,user_id,song1,song2,song3,song4,song5,song6,song7,song8,song9,song10
0,0011d5f4fb02ff276763d385c3f2ded2b00ad94a,SOTNWCI12AAF3B2028,SOXIDBV12AAF3B3B76,SOVCFPI12AAF3B2045,SOOBSJA12AAA8C7724,SOETFHY12AAF3B3FD4,SOUATZO12AAF3B23A7,SOMBCOW12AAF3B229F,SOZWZEH12AAF3B2578,SOFCKRY12AAF3B2FAF,SOLVEAC12AAF3B2DC2
1,00a80e20a8b269747ad5f511fb579ef9ec013941,SOGAUIQ12A6D4F8262,SOZIMLB12AF72AD1CD,SOUHRQN12A58A7C12A,SONCYFS12A8C14261E,SOPYZLU12A8C14193C,SOCZSYW12AAF3B490E,SOBOAQC12A8C13E3E9,SOISAFU12CF5F882E5,SOULMYP12A8C143D8B,SOWCQMB12A8C141825
2,0107197bd5e2521cfa1cfde3faf9a6b39e8ba9e3,SOIHJSD12A6701EB04,SOGMROZ12A679D8AE9,SOFOYJY12A679D30E3,SOXVSTI12A6310ECC7,SONQNPM12A67020A32,SOXYGNC12A67021C29,SOPOFBK12AF729EEBE,SOYGIBP12A67020AF7,SOQUGRT12A6701DD47,SOJCBAM12A6701FD04
3,0118d4558afda9bdaa39ec3ea3dbe62f12604d4d,SOMBWDW12A6D4F7248,SOEKSLB12A6D4F56FE,SOHTPZM12A6D4F8716,SOIMTBN12A6D4F56FA,SOCNGFJ12A6D4F6DFD,SORCHVC12A6D4F7651,SOMTYWT12A58A75BF7,SOHHVXI12A6D4F69E3,SOIJZUS12A6D4F73B1,SOCZJSH12A6D4F7636
4,0160c5e9f3c439d601d379ea0d284e3979b4fc19,SOKZILC12A6D4FBC9D,SOZEPNS12A8AE4766D,SORAACY12A8AE475D2,SOUBALJ12A8AE46A0B,SOOXNKG12A58A7C6BC,SOOVLKO12A58A7F7BA,SOESGIR12A8AE4855D,SOGXSWA12A6D4FBC99,SOFHWUZ12A8AE48B9D,SOTRAIA12A8AE47465
5,026db54b18e6e1c8273d82f2f6b1f1c81676d01b,SOWXKLQ12A8C13C221,SOSLEMJ12A8C13F86B,SOKQDCA12A8C13CDEF,SOUSQZF12A8C13BBB9,SOVOWRW12A8C13B93D,SOHBATX12A8C14009F,SORJVDO12AF72A1970,SOIQFYV12A8C13EEB6,SOMDTTT12A8C13F1CB,SOKQDCS12AF72A920F
6,02caaa0349f5d7aa54a6a262308a4ada43ad46d2,SOCWJDB12A58A776AF,SONERDT12AB017EE72,SOVSNJX12AB0189848,SOONBLX12AC468AA6B,SOLWZMY12AB0182AAD,SOUQZLK12AC468B143,SOFKPLB12AC95F0493,SORRNTH12AC960A43B,SOBUVXZ12AC9075605,SOWTJRG12AC9075799
7,02e4af8e545c96c98359bc875a2db8f1e28bc42b,SOPDFZN12A670215C4,SOUJKAH12A6701F2A5,SOGDAUG12A6701F302,SOGIUWE12A670215DC,SOSBPUO12A58291CF2,SOGPEMQ12A67020412,SOBXZGV12A6702074F,SOFGAJF12A8C141DA3,SOQZZKI12A67020D38,SOPIHQU12A670200C6
8,03ace7391031faa7d9b152f39fe78cb57e0f078c,SOWVTCS12AC468543B,SOEVOMG12AC4688558,SOYBGAI12AC468AA34,SOPNFYY12AB01828D4,SOACEXG12AC468D8FB,SOHFZQQ12A58A7FA07,SOGXKOP12A58A7FA1C,SOMQNVZ12AC468A9AD,SOVDNXZ12AC3DF9EA4,SOYYHWG12AC3DF9F08
9,03b9807f06e6843382f1443ce56d9e32dd087e0c,SOULTKQ12AB018A183,SOKFHLV12AB0187A2F,SOSLLDN12AB018CA89,SOZXLAE12AB018AB74,SOROOOR12AB018572F,SOKRHQK12AB018CCC0,SOTCXIY12AB0183D7A,SOHBIKA12AB01857DB,SOCJPYJ12AB0188BCC,SOBACBP12AB0188B9F


In [65]:
keydict = {}
for user, song in zip(trainingdata.userid, trainingdata.songid):
    try:
        keydict[user].append(song)
    except KeyError:
        keydict[user] = []
        keydict[user].append(song)

In [67]:
reco = {}
for user, song1 in zip(recommendations.user_id, recommendations.song1):
    try:
        reco[user].append(song1)
    except KeyError:
        reco[user] = []
        reco[user].append(song1)
        
for user, song2 in zip(recommendations.user_id, recommendations.song2):
    try:
        reco[user].append(song2)
    except KeyError:
        reco[user] = []
        reco[user].append(song2)
        
for user, song3 in zip(recommendations.user_id, recommendations.song3):
    try:
        reco[user].append(song3)
    except KeyError:
        reco[user] = []
        reco[user].append(song3)
        
for user, song4 in zip(recommendations.user_id, recommendations.song4):
    try:
        reco[user].append(song4)
    except KeyError:
        reco[user] = []
        reco[user].append(song4)


for user, song5 in zip(recommendations.user_id, recommendations.song5):
    try:
        reco[user].append(song5)
    except KeyError:
        reco[user] = []
        reco[user].append(song5)
        
        
for user, song6 in zip(recommendations.user_id, recommendations.song6):
    try:
        reco[user].append(song6)
    except KeyError:
        reco[user] = []
        reco[user].append(song6)
        
for user, song7 in zip(recommendations.user_id, recommendations.song7):
    try:
        reco[user].append(song7)
    except KeyError:
        reco[user] = []
        reco[user].append(song7)
        
for user, song8 in zip(recommendations.user_id, recommendations.song8):
    try:
        reco[user].append(song8)
    except KeyError:
        reco[user] = []
        reco[user].append(song8)
        
for user, song9 in zip(recommendations.user_id, recommendations.song9):
    try:
        reco[user].append(song9)
    except KeyError:
        reco[user] = []
        reco[user].append(song9)
        
for user, song10 in zip(recommendations.user_id, recommendations.song10):
    try:
        reco[user].append(song10)
    except KeyError:
        reco[user] = []
        reco[user].append(song10)

In [68]:
match = 0
nonmatch = 0
count = 0
for i in reco:
    recoset = set(reco[i])
    if i in keydict:
        keydictset = set(keydict[i])
        matches = (recoset - (recoset - keydictset))
        match += len(matches)
        nonmatch += 10 - len(matches)

In [69]:
match = float(match)
nonmatch = float(nonmatch)

In [70]:
totalreco = recommendations.shape[0]*(recommendations.shape[1]-1)
totalreco

In [73]:
precision = float(match/(match+nonmatch))
precision

0.08698010849909583

In [74]:
songs_sample = songs.set_index("song.id")

In [75]:
mylist = []
for i in range(1,len(recommendations.ix[0])):
    mylist.append(songs_sample.ix[recommendations.ix[0][i]]["title"])
    
samplerecommendation = pd.DataFrame(data = mylist)
samplerecommendation = samplerecommendation.rename(index=str, columns={0: recommendations.ix[0][0]})

In [76]:
samplerecommendation

,0011d5f4fb02ff276763d385c3f2ded2b00ad94a
0,The Way I Loved You
1,Nostradamus
2,Should've Said No
3,Jack and Benton
4,Knight -> King 4
5,When I Get Home
6,On Efface
7,Pai Keros
8,Rainin' In My Heart
9,Pee Wee's Boogie


#### Reference

https://github.com/prachi108/Million-Song-Project